## Load Dataset

In [2]:
import os
import pandas as pd
from scipy.stats import zscore
import matplotlib as matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [3]:
df = pd.read_csv("data\Occupancy.csv")
df.head().T

,0,1,2,3,4
date,2015-02-02 14:19:00,2015-02-02 14:19:59,2015-02-02 14:21:00,2015-02-02 14:22:00,2015-02-02 14:23:00
Temperature,23.7,23.718,23.73,23.7225,23.754
Humidity,26.272,26.29,26.23,26.125,26.2
Light,585.2,578.4,572.666667,493.75,488.6
CO2,749.2,760.4,769.666667,774.75,779.0
HumidityRatio,0.004764,0.004773,0.004765,0.004744,0.004767
Occupancy,1,1,1,1,1


# Check columns and types in dataset

In [4]:
df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20560 entries, 0 to 20559
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           20560 non-null  object 
 1   Temperature    20560 non-null  float64
 2   Humidity       20560 non-null  float64
 3   Light          20560 non-null  float64
 4   CO2            20560 non-null  float64
 5   HumidityRatio  20560 non-null  float64
 6   Occupancy      20560 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.1+ MB


In [5]:
df.shape

(20560, 7)

# Cleaning the dataset

### Check for missing values

In [6]:
df.isnull().sum()

date             0
Temperature      0
Humidity         0
Light            0
CO2              0
HumidityRatio    0
Occupancy        0
dtype: int64

### Check for duplicates

In [7]:
df['date'].duplicated().sum() 

np.int64(0)

# Look at basic statistics

In [8]:
df.describe()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
count,20560.000000,20560.000000,20560.000000,20560.000000,20560.000000,20560.000000
mean,20.906212,27.655925,130.756622,690.553276,0.004228,0.231031
std,1.055315,4.982154,210.430875,311.201281,0.000768,0.421503
min,19.000000,16.745000,0.000000,412.750000,0.002674,0.000000
25%,20.200000,24.500000,0.000000,460.000000,0.003719,0.000000
50%,20.700000,27.290000,0.000000,565.416667,0.004292,0.000000
75%,21.525000,31.290000,301.000000,804.666667,0.004832,0.000000
max,24.408333,39.500000,1697.250000,2076.500000,0.006476,1.000000


## Look at dates in dataset

In [ ]:
# Convert to datetime if not already
df['date'] = pd.to_datetime(df['date'])

# Check min and max timestamps
start_date = df['date'].min()
end_date = df['date'].max()

print(f"📅 Data ranges from: {start_date} to {end_date}")
print(f"🕒 Total duration: {end_date - start_date}")


📅 Data ranges from: 2015-02-02 14:19:00 to 2015-02-18 09:19:00
🕒 Total duration: 15 days 19:00:00


In [ ]:
df['date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 20560 entries, 0 to 20559
Series name: date
Non-Null Count  Dtype 
--------------  ----- 
20560 non-null  object
dtypes: object(1)
memory usage: 160.8+ KB


## 🏠 Indoor Averages in February (Europe) ground truth for columns is:
#### 💧 Humidity: 30% – 40% Relative Humidity
#### 💡 Light: 300–500 Lux
#### 🫁 CO2: ~426.13 ppm
#### 🌡️ Temprature: 18°C to 22°C (vs. outdoors: 8°C to 14°C)
#### ⚖️ Humidity Ratio: ~0.0045 kg water/kg dry air

#### -> sources: REHVA, ASHRAE, ScienceDirect, LEOTEK, Euoropean Enviroment Agency

# Looking at different correlations - Answering Questions

### Occupancy Distribution

In [ ]:
import plotly.express as px

fig = px.histogram(
    df,
    x='Occupancy',
    nbins=2,
    histnorm='percent',
    title='Occupancy Distribution (Percentage)'
)
fig.update_layout(
    xaxis_title='Occupancy',
    yaxis_title='Percentage',
    bargap=0.2
)
fig.show()


### Compare occupancy for day-off or working day


In [ ]:
import pandas as pd
import plotly.express as px

# Ensure datetime format and extract hour
df['date'] = pd.to_datetime(df['date'])
df['hour'] = df['date'].dt.hour
df['day'] = df['date'].dt.day_name()

# Filter for Sunday and Monday
df_filtered = df[df['day'].isin(['Sunday', 'Monday'])]

# Group by day and hour, calculate average occupancy
occupancy_by_hour = df_filtered.groupby(['day', 'hour'])['Occupancy'].mean().reset_index()

# Plot
fig = px.line(
    occupancy_by_hour,
    x='hour',
    y='Occupancy',
    color='day',
    title='Average Occupancy by Hour: Sunday vs. Monday',
    markers=True,
    labels={'hour': 'Hour of Day', 'Occupancy': 'Occupancy Rate'}
)

fig.update_layout(
    xaxis=dict(dtick=1),
    yaxis=dict(range=[0, 1]),
    template='plotly_white'
)

fig.show()


### Conclusion: Sunday is day - off

### Occupancy vs Temprature

In [ ]:
import plotly.express as px

fig = px.box(
    df,
    x='Occupancy',
    y='Temperature',
    title='Temperature Distribution by Occupancy Status',
    labels={'Occupancy': 'Occupancy (0 = No, 1 = Yes)', 'Temperature': 'Temperature (°C)'}
)
fig.show()


In [ ]:
# Convert 'date' to datetime and extract hour
df['date'] = pd.to_datetime(df['date'])
df['hour'] = df['date'].dt.hour

# Group by hour
hourly_stats = df.groupby('hour').agg({
    'Temperature': 'mean',
    'Occupancy': 'mean'  # average occupancy per hour (0 to 1)
}).reset_index()

# Create the figure
fig = go.Figure()

# Add temperature trace (left y-axis)
fig.add_trace(go.Scatter(
    x=hourly_stats['hour'],
    y=hourly_stats['Temperature'],
    name='Avg Temperature (°C)',
    mode='lines+markers',
    line=dict(color='red'),
    yaxis='y1'
))

# Add occupancy trace (right y-axis)
fig.add_trace(go.Bar(
    x=hourly_stats['hour'],
    y=hourly_stats['Occupancy'],
    name='Avg Occupancy',
    opacity=0.3,
    marker_color='blue',
    yaxis='y2'
))

# Update layout
fig.update_layout(
    title='Average Temperature and Occupancy by Hour of Day',
    xaxis=dict(title='Hour of Day', dtick=1),
    yaxis=dict(
        title='Avg Temperature (°C)',
        side='left',
        showgrid=False
    ),
    yaxis2=dict(
        title='Avg Occupancy',
        side='right',
        overlaying='y',
        range=[0, 1],
        showgrid=False
    ),
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

fig.show()


### Compare temprature for a day off and a working day

In [21]:
# Ensure 'date' is datetime
df['date'] = pd.to_datetime(df['date'])

# Extract hour
df['hour'] = df['date'].dt.hour

# Filter for Sunday and hours between 8 and 17 inclusive
df_sunday_workhours = df[
    (df['date'].dt.day_name() == 'Sunday') &
    (df['hour'] >= 8) &
    (df['hour'] <= 17)
]

# Calculate average temperature
average_temp_sunday_workhours = df_sunday_workhours['Temperature'].mean()
print(f"🌡️ Average Temperature on Sunday (08:00–17:00): {average_temp_sunday_workhours:.2f} °C")

# Filter for Monday and hours between 8 and 17 inclusive
df_monday_workhours = df[
    (df['date'].dt.day_name() == 'Monday') &
    (df['hour'] >= 8) &
    (df['hour'] <= 17)
]

# Calculate average temperature
average_temp_monday_workhours = df_monday_workhours['Temperature'].mean()
print(f"🌡️ Average Temperature on Monday (08:00–17:00): {average_temp_monday_workhours:.2f} °C")


🌡️ Average Temperature on Sunday (08:00–17:00): 20.67 °C
🌡️ Average Temperature on Monday (08:00–17:00): 21.62 °C


### Average CO₂ and Occupancy by Hour of Day

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Convert 'date' to datetime and extract hour
df['date'] = pd.to_datetime(df['date'])
df['hour'] = df['date'].dt.hour

# Group by hour
hourly_stats = df.groupby('hour').agg({
    'CO2': 'mean',
    'Occupancy': 'mean'  # average occupancy per hour (0 to 1)
}).reset_index()

# Create the figure
fig = go.Figure()

# Add CO2 trace (left y-axis)
fig.add_trace(go.Scatter(
    x=hourly_stats['hour'],
    y=hourly_stats['CO2'],
    name='Avg CO₂ (ppm)',
    mode='lines+markers',
    line=dict(color='green'),
    yaxis='y1'
))

# Add occupancy trace (right y-axis)
fig.add_trace(go.Bar(
    x=hourly_stats['hour'],
    y=hourly_stats['Occupancy'],
    name='Avg Occupancy',
    opacity=0.3,
    marker_color='blue',
    yaxis='y2'
))

# Update layout
fig.update_layout(
    title='Average CO₂ and Occupancy by Hour of Day',
    xaxis=dict(title='Hour of Day', dtick=1),
    yaxis=dict(
        title='Avg CO₂ (ppm)',
        side='left',
        showgrid=False
    ),
    yaxis2=dict(
        title='Avg Occupancy',
        side='right',
        overlaying='y',
        range=[0, 1],
        showgrid=False
    ),
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

fig.show()


### Compare temprature for a day off and a working day

In [ ]:
# Filter work hours for Sunday
df_sun_work = df[
    (df['day'] == 'Sunday') &
    (df['hour'] >= 8) & (df['hour'] <= 17)
]
avg_co2_sun = df_sun_work['CO2'].mean()
print(f"🌿 Average CO₂ on Sunday (08:00–17:00): {avg_co2_sun:.2f} ppm")

# Filter work hours for Monday
df_mon_work = df[
    (df['day'] == 'Monday') &
    (df['hour'] >= 8) & (df['hour'] <= 17)
]
avg_co2_mon = df_mon_work['CO2'].mean()
print(f"🌿 Average CO₂ on Monday (08:00–17:00): {avg_co2_mon:.2f} ppm")


🌿 Average CO₂ on Sunday (08:00–17:00): 536.98 ppm
🌿 Average CO₂ on Monday (08:00–17:00): 1110.60 ppm


### Humidity and Occupancy by Hour

In [ ]:
# Group by hour for Humidity and Occupancy
hourly_humidity = df.groupby('hour').agg({
    'Humidity': 'mean',
    'Occupancy': 'mean'
}).reset_index()

# Create figure
fig = go.Figure()

# Add Humidity trace (left Y-axis)
fig.add_trace(go.Scatter(
    x=hourly_humidity['hour'],
    y=hourly_humidity['Humidity'],
    name='Avg Humidity (%)',
    mode='lines+markers',
    line=dict(color='orange'),
    yaxis='y1'
))

# Add Occupancy trace (right Y-axis)
fig.add_trace(go.Bar(
    x=hourly_humidity['hour'],
    y=hourly_humidity['Occupancy'],
    name='Avg Occupancy',
    opacity=0.3,
    marker_color='blue',
    yaxis='y2'
))

# Layout
fig.update_layout(
    title='Average Humidity and Occupancy by Hour of Day',
    xaxis=dict(title='Hour of Day', dtick=1),
    yaxis=dict(title='Avg Humidity (%)', side='left'),
    yaxis2=dict(title='Avg Occupancy', side='right', overlaying='y', range=[0, 1]),
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

fig.show()


### HumidityRatio and Occupancy by Hour

In [ ]:
# Group by hour for HumidityRatio and Occupancy
hourly_ratio = df.groupby('hour').agg({
    'HumidityRatio': 'mean',
    'Occupancy': 'mean'
}).reset_index()

# Create figure
fig = go.Figure()

# Add HumidityRatio trace (left Y-axis)
fig.add_trace(go.Scatter(
    x=hourly_ratio['hour'],
    y=hourly_ratio['HumidityRatio'],
    name='Avg Humidity Ratio (kg/kg)',
    mode='lines+markers',
    line=dict(color='purple'),
    yaxis='y1'
))

# Add Occupancy trace (right Y-axis)
fig.add_trace(go.Bar(
    x=hourly_ratio['hour'],
    y=hourly_ratio['Occupancy'],
    name='Avg Occupancy',
    opacity=0.3,
    marker_color='blue',
    yaxis='y2'
))

# Layout
fig.update_layout(
    title='Average Humidity Ratio and Occupancy by Hour of Day',
    xaxis=dict(title='Hour of Day', dtick=1),
    yaxis=dict(title='Avg Humidity Ratio (kg/kg)', side='left'),
    yaxis2=dict(title='Avg Occupancy', side='right', overlaying='y', range=[0, 1]),
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

fig.show()


### Correlation between time and light throughout the day (light mean x hour of day)

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Ensure datetime format and extract hour
df['date'] = pd.to_datetime(df['date'])
df['hour'] = df['date'].dt.hour

# Group by hour and calculate mean Light and Occupancy
hourly_stats = df.groupby('hour').agg({
    'Light': 'mean',
    'Occupancy': 'mean'  
}).reset_index()

# Create figure
fig = go.Figure()

# Add Occupancy (background area)
fig.add_trace(go.Bar(
    x=hourly_stats['hour'],
    y=hourly_stats['Occupancy'],
    name='Occupancy Rate',
    opacity=0.3,
    marker_color='lightblue',
    yaxis='y2'
))

# Add Light (line plot)
fig.add_trace(go.Scatter(
    x=hourly_stats['hour'],
    y=hourly_stats['Light'],
    name='Avg Light',
    mode='lines+markers',
    line=dict(color='orange'),
    yaxis='y1'
))

# Update layout with two y-axes
fig.update_layout(
    title='Average Light Levels with Occupancy by Hour of Day',
    xaxis=dict(title='Hour of Day', dtick=1),
    yaxis=dict(
        title='Average Light',
        side='left'
    ),
    yaxis2=dict(
        title='Occupancy Rate',
        overlaying='y',
        side='right',
        range=[0, 1],
        showgrid=False
    ),
    template='plotly_white',
    legend=dict(x=0.01, y=0.99)
)

fig.show()


### Check for real Sunset distribution in europe

#### sunrise : 7-8 , sunset: 17-18

### Check Light distribution for a day off (sunday)

In [ ]:
import pandas as pd
import plotly.express as px

# Ensure datetime format
df['date'] = pd.to_datetime(df['date'])

# Filter for Sunday (weekday == 6 in pandas, or use .day_name())
df_sunday = df[df['date'].dt.day_name() == 'Sunday']

# Extract hour
df_sunday['hour'] = df_sunday['date'].dt.hour

# Group by hour and calculate average light
hourly_light_sunday = df_sunday.groupby('hour')['Light'].mean().reset_index()

# Plot
fig = px.line(
    hourly_light_sunday,
    x='hour',
    y='Light',
    title='Average Light Levels by Hour on Sunday',
    markers=True
)
fig.update_layout(
    xaxis_title='Hour of Day',
    yaxis_title='Average Light',
    xaxis=dict(dtick=1),
    template='plotly_white'
)
fig.show()


C:\Users\inbal\AppData\Local\Temp\ipykernel_16244\3338015965.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
# Filter only Sunday
df_sunday = df[df['date'].dt.day_name() == 'Sunday']

# Calculate average light on Sunday
average_light_sunday = df_sunday['Light'].mean()
print(f"🌞 Average Light on Sunday: {average_light_sunday:.2f}")

# Filter only Monday
df_monday = df[df['date'].dt.day_name() == 'Monday']

# Calculate average light on Monday
average_light_monday = df_monday['Light'].mean()
print(f"🌞 Average Light on Monday: {average_light_monday:.2f}")


🌞 Average Light on Sunday: 38.74
🌞 Average Light on Monday: 172.32


# Check outliers

### Select only float-type columns (continuous numeric data)

In [ ]:
numeric_cols = df.select_dtypes(include=['float32', 'float64']).columns
numeric_cols 

Index(['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio'], dtype='object')

#### Temprature:

In [ ]:
col = 'Temperature'

# IQR Method
Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

iqr_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"🔹 IQR outliers in '{col}': {len(iqr_outliers)}")

# Optional: collect into a DataFrame
if not iqr_outliers.empty:
    df_iqr_outliers = iqr_outliers[[col]].copy()
    df_iqr_outliers["column"] = col
    df_iqr_outliers["method"] = "IQR"
else:
    df_iqr_outliers = pd.DataFrame()


🔹 IQR outliers in 'Temperature': 519


#### Humidity:

In [ ]:
col = 'Humidity'

# IQR Method
Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

iqr_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"🔹 IQR outliers in '{col}': {len(iqr_outliers)}")

# Optional: collect into a DataFrame
if not iqr_outliers.empty:
    df_iqr_outliers = iqr_outliers[[col]].copy()
    df_iqr_outliers["column"] = col
    df_iqr_outliers["method"] = "IQR"
else:
    df_iqr_outliers = pd.DataFrame()


🔹 IQR outliers in 'Humidity': 0


#### Light:

In [ ]:
col = 'Light'

# IQR Method
Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

iqr_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"🔹 IQR outliers in '{col}': {len(iqr_outliers)}")

# Optional: collect into a DataFrame
if not iqr_outliers.empty:
    df_iqr_outliers = iqr_outliers[[col]].copy()
    df_iqr_outliers["column"] = col
    df_iqr_outliers["method"] = "IQR"
else:
    df_iqr_outliers = pd.DataFrame()


🔹 IQR outliers in 'Light': 183


#### CO2:

In [ ]:
col = 'CO2'

# IQR Method
Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

iqr_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"🔹 IQR outliers in '{col}': {len(iqr_outliers)}")

# Optional: collect into a DataFrame
if not iqr_outliers.empty:
    df_iqr_outliers = iqr_outliers[[col]].copy()
    df_iqr_outliers["column"] = col
    df_iqr_outliers["method"] = "IQR"
else:
    df_iqr_outliers = pd.DataFrame()


🔹 IQR outliers in 'CO2': 1449


#### Humidity Ratio:

In [ ]:
col = 'HumidityRatio'

# IQR Method
Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

iqr_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"🔹 IQR outliers in '{col}': {len(iqr_outliers)}")

# Optional: collect into a DataFrame
if not iqr_outliers.empty:
    df_iqr_outliers = iqr_outliers[[col]].copy()
    df_iqr_outliers["column"] = col
    df_iqr_outliers["method"] = "IQR"
else:
    df_iqr_outliers = pd.DataFrame()


🔹 IQR outliers in 'HumidityRatio': 0
